# 🔬 Optimisation Multi-Stratégies : Envelope Parameters

Ce notebook optimise les **paramètres de trading** de la stratégie multi-envelope :
- `ma_base_window` : Réactivité du signal
- `envelopes` : Distance d'entrée
- `size` : Risque par trade  
- `stop_loss` : Protection

**Méthodologie** : Walk-Forward Optimization avec Expanding Window pour éviter l'overfitting.

**⚠️ Important** : L'optimisation de la détection de régime se fait dans `multi_envelope_adaptive.ipynb`.

## 1️⃣ Configuration et chargement des données

In [1]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed  # Pour multi-core

import warnings
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

from indicator_cache import IndicatorCache, precompute_all_indicators
from optimized_worker import prepare_data_for_worker

# Backtest engine
from utilities.strategies.envelopeMulti_v2 import EnvelopeMulti_v2
from utilities.data_manager import ExchangeDataManager

# Système adaptatif
from core import calculate_regime_series, DEFAULT_PARAMS
from core.params_adapter import FixedParamsAdapter, RegimeBasedAdapter
from core.backtest_comparator import BacktestComparator

# === ETAPE 2A : CHARGEMENT MAPPING NB ENVELOPES ===
import os
from pathlib import Path

envelope_mapping_path = 'envelope_count_mapping.csv'  # Même dossier que le notebook

if os.path.exists(envelope_mapping_path):
    df_envelope_mapping = pd.read_csv(envelope_mapping_path, index_col='pair')
    print(f"✅ Mapping nb envelopes chargé : {len(df_envelope_mapping)} pairs")
    print(f"   4 envelopes : {(df_envelope_mapping['n_envelopes'] == 4).sum()} pairs")
    print(f"   3 envelopes : {(df_envelope_mapping['n_envelopes'] == 3).sum()} pairs")
else:
    print("⚠️  WARNING: envelope_count_mapping.csv non trouvé")
    print("   Exécutez d'abord : python assign_envelope_count.py")
    df_envelope_mapping = None

# Config plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Imports réussis (multi-core activé)")

c:\Users\jacks\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Module optimized_worker chargé (numpy views + mémoire optimisée)
✅ Mapping nb envelopes chargé : 28 pairs
   4 envelopes : 7 pairs
   3 envelopes : 21 pairs
✅ Imports réussis (multi-core activé)


In [2]:
# ======================
# CONFIGURATION GLOBALE
# ======================

BACKTEST_LEVERAGE = 10
INITIAL_WALLET = 1000
EXCHANGE = "binance"
TEST_MODE = True

# Périodes (Expanding Window)
# Couvre tous les cycles: BULL 2020-2021, BEAR 2022, RECOVERY 2023, BULL 2024, BULL 2025
PERIODS = {
    "train_full": {"start": "2020-01-01", "end": "2025-06-30"},  # Optimisation (toute la période sauf 3 mois)
    "holdout": {"start": "2025-07-01", "end": "2025-10-03"},     # Validation finale (3 derniers mois)
}

# Walk-Forward Folds (Expanding Window)
if TEST_MODE:
    # 🧪 MODE TEST : 2 folds uniquement (données 2024-2025)
    WF_FOLDS = [
        {"train_start": "2024-01-01", "train_end": "2024-06-30", "test_start": "2024-07-01", "test_end": "2024-12-31", "name": "Fold1_TEST"},
        {"train_start": "2024-01-01", "train_end": "2024-12-31", "test_start": "2025-01-01", "test_end": "2025-06-30", "name": "Fold2_TEST"},
    ]
else:
    # ✅ MODE PRODUCTION : 7 folds complets
    WF_FOLDS = [
        {"train_start": "2020-01-01", "train_end": "2021-12-31", "test_start": "2022-01-01", "test_end": "2022-06-30", "name": "Fold1_Bull2020-21→Bear2022"},
        {"train_start": "2020-01-01", "train_end": "2022-06-30", "test_start": "2022-07-01", "test_end": "2022-12-31", "name": "Fold2_Bull+Bear→Bear"},
        {"train_start": "2020-01-01", "train_end": "2022-12-31", "test_start": "2023-01-01", "test_end": "2023-06-30", "name": "Fold3_Full→Recovery"},
        {"train_start": "2020-01-01", "train_end": "2023-06-30", "test_start": "2023-07-01", "test_end": "2023-12-31", "name": "Fold4_Recovery→Bull2023"},
        {"train_start": "2020-01-01", "train_end": "2023-12-31", "test_start": "2024-01-01", "test_end": "2024-06-30", "name": "Fold5_Full→Bull2024-H1"},
        {"train_start": "2020-01-01", "train_end": "2024-06-30", "test_start": "2024-07-01", "test_end": "2024-12-31", "name": "Fold6_2020-24-H1→Bull2024-H2"},
        {"train_start": "2020-01-01", "train_end": "2024-12-31", "test_start": "2025-01-01", "test_end": "2025-06-30", "name": "Fold7_2020-24→Bull2025-H1"},
    ]

# Échantillon stratifié de paires représentatives
if TEST_MODE:
    # 🧪 MODE TEST : 4 paires (1 par profil)
    PAIRS = [
        "BTC/USDT:USDT",   # Major
        "SOL/USDT:USDT",   # Mid-cap
        "DOGE/USDT:USDT",  # Volatile
        "TRX/USDT:USDT",   # Low performer
    ]
else:
    # ✅ MODE PRODUCTION : 8 paires complètes
    PAIRS = [
        "BTC/USDT:USDT",   # Major
        "ETH/USDT:USDT",   # Major
        "SOL/USDT:USDT",   # Mid-cap
        "AVAX/USDT:USDT",  # Mid-cap
        "ADA/USDT:USDT",   # Mid-cap
        "DOGE/USDT:USDT",  # Volatile
        "SUSHI/USDT:USDT", # Volatile (meilleur performer historique)
        "TRX/USDT:USDT",   # Low performer
    ]

# Classification par profil
PAIR_CLASSES = {
    "BTC/USDT:USDT": "major",
    "ETH/USDT:USDT": "major",
    "SOL/USDT:USDT": "mid-cap",
    "AVAX/USDT:USDT": "mid-cap",
    "ADA/USDT:USDT": "mid-cap",
    "DOGE/USDT:USDT": "volatile",
    "SUSHI/USDT:USDT": "volatile",
    "TRX/USDT:USDT": "low",
}

# Paramètres de backtest communs
BACKTEST_PARAMS = {
    "initial_wallet": INITIAL_WALLET,
    "leverage": BACKTEST_LEVERAGE,
    "maker_fee": 0.0002,
    "taker_fee": 0.0006,
    "reinvest": True,
    "liquidation": True,
    "risk_mode": "scaling",
}

print(f"✅ Configuration chargée")
if TEST_MODE:
    print(f"   🧪 MODE TEST RAPIDE")
    print(f"   Folds: {len(WF_FOLDS)} (au lieu de 7)")
    print(f"   Paires: {len(PAIRS)} (au lieu de 8)")
else:
    print(f"   Période: {PERIODS['train_full']['start']} → {PERIODS['holdout']['end']} (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")
    print(f"   Paires: {len(PAIRS)} (échantillon stratifié)")
print(f"   - Majors: {sum(1 for p in PAIRS if PAIR_CLASSES.get(p) == 'major')}")
print(f"   - Mid-caps: {sum(1 for p in PAIRS if PAIR_CLASSES.get(p) == 'mid-cap')}")
print(f"   - Volatiles: {sum(1 for p in PAIRS if PAIR_CLASSES.get(p) == 'volatile')}")
print(f"   - Low performers: {sum(1 for p in PAIRS if PAIR_CLASSES.get(p) == 'low')}")
print(f"   Walk-Forward Folds: {len(WF_FOLDS)}")
if not TEST_MODE:
    print(f"   Hold-out: {PERIODS['holdout']['start']} → {PERIODS['holdout']['end']}")

✅ Configuration chargée
   🧪 MODE TEST RAPIDE
   Folds: 2 (au lieu de 7)
   Paires: 4 (au lieu de 8)
   - Majors: 1
   - Mid-caps: 1
   - Volatiles: 1
   - Low performers: 1
   Walk-Forward Folds: 2


In [3]:
# ======================
# CONFIGURATION GLOBALE
# ======================

BACKTEST_LEVERAGE = 10
INITIAL_WALLET = 1000
EXCHANGE = "binance"

# Périodes (Expanding Window)
# Couvre tous les cycles: BULL 2020-2021, BEAR 2022, RECOVERY 2023, BULL 2024, BULL 2025
PERIODS = {
    "train_full": {"start": "2020-01-01", "end": "2025-06-30"},  # Optimisation (toute la période sauf 3 mois)
    "holdout": {"start": "2025-07-01", "end": "2025-10-03"},     # Validation finale (3 derniers mois)
}

# Walk-Forward Folds (Expanding Window)
WF_FOLDS = [
    {"train_start": "2020-01-01", "train_end": "2021-12-31", "test_start": "2022-01-01", "test_end": "2022-06-30", "name": "Fold1_Bull2020-21→Bear2022"},
    {"train_start": "2020-01-01", "train_end": "2022-06-30", "test_start": "2022-07-01", "test_end": "2022-12-31", "name": "Fold2_Bull+Bear→Bear"},
    {"train_start": "2020-01-01", "train_end": "2022-12-31", "test_start": "2023-01-01", "test_end": "2023-06-30", "name": "Fold3_Full→Recovery"},
    {"train_start": "2020-01-01", "train_end": "2023-06-30", "test_start": "2023-07-01", "test_end": "2023-12-31", "name": "Fold4_Recovery→Bull2023"},
    {"train_start": "2020-01-01", "train_end": "2023-12-31", "test_start": "2024-01-01", "test_end": "2024-06-30", "name": "Fold5_Full→Bull2024-H1"},
    {"train_start": "2020-01-01", "train_end": "2024-06-30", "test_start": "2024-07-01", "test_end": "2024-12-31", "name": "Fold6_2020-24-H1→Bull2024-H2"},
    {"train_start": "2020-01-01", "train_end": "2024-12-31", "test_start": "2025-01-01", "test_end": "2025-06-30", "name": "Fold7_2020-24→Bull2025-H1"},
]

# Échantillon stratifié de 8 paires représentatives
# (couvre majors, mid-caps, volatiles, low performers)
PAIRS = [
    "BTC/USDT:USDT",   # Major
    "ETH/USDT:USDT",   # Major
    "SOL/USDT:USDT",   # Mid-cap
    "AVAX/USDT:USDT",  # Mid-cap
    "ADA/USDT:USDT",   # Mid-cap
    "DOGE/USDT:USDT",  # Volatile
    "SUSHI/USDT:USDT", # Volatile (meilleur performer historique)
    "TRX/USDT:USDT",   # Low performer
]

# Classification par profil
PAIR_CLASSES = {
    "BTC/USDT:USDT": "major",
    "ETH/USDT:USDT": "major",
    "SOL/USDT:USDT": "mid-cap",
    "AVAX/USDT:USDT": "mid-cap",
    "ADA/USDT:USDT": "mid-cap",
    "DOGE/USDT:USDT": "volatile",
    "SUSHI/USDT:USDT": "volatile",
    "TRX/USDT:USDT": "low",
}

# Paramètres de backtest communs
BACKTEST_PARAMS = {
    "initial_wallet": INITIAL_WALLET,
    "leverage": BACKTEST_LEVERAGE,
    "maker_fee": 0.0002,
    "taker_fee": 0.0006,
    "reinvest": True,
    "liquidation": True,
    "risk_mode": "scaling",
}

print(f"✅ Configuration chargée")
print(f"   Période: {PERIODS['train_full']['start']} → {PERIODS['holdout']['end']} (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")
print(f"   Paires: {len(PAIRS)} (échantillon stratifié)")
print(f"   - Majors: {sum(1 for c in PAIR_CLASSES.values() if c == 'major')}")
print(f"   - Mid-caps: {sum(1 for c in PAIR_CLASSES.values() if c == 'mid-cap')}")
print(f"   - Volatiles: {sum(1 for c in PAIR_CLASSES.values() if c == 'volatile')}")
print(f"   - Low performers: {sum(1 for c in PAIR_CLASSES.values() if c == 'low')}")
print(f"   Walk-Forward Folds: {len(WF_FOLDS)}")
print(f"   Hold-out: {PERIODS['holdout']['start']} → {PERIODS['holdout']['end']}")

✅ Configuration chargée
   Période: 2020-01-01 → 2025-10-03 (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)
   Paires: 8 (échantillon stratifié)
   - Majors: 2
   - Mid-caps: 3
   - Volatiles: 2
   - Low performers: 1
   Walk-Forward Folds: 7
   Hold-out: 2025-07-01 → 2025-10-03


In [5]:
# === CELL-3b : GRILLES PAR PROFIL (MULTIPLICATEURS + NB ENV AUTO) ===

# Config globale de référence (issue de l'optimisation Étape 1)
BASE_CONFIG = {
    'ma_base_window': 5,           # Meilleure MA de l'optimisation globale
    'envelopes_3': [0.07, 0.1, 0.15],     # Base pour 3 envelopes
    'envelopes_4': [0.07, 0.1, 0.15, 0.20],  # Base pour 4 envelopes
    'size': 0.12,                  # Meilleur size de l'optimisation globale
    'stop_loss': 0.25
}

# Multiplicateurs par profil (au lieu de valeurs absolues)
# Objectif : Réduire l'espace de recherche tout en adaptant aux volatilités
PROFILE_MULTIPLIERS = {
    "major": {
        "mult": [0.8, 0.9, 1.0],        # BTC/ETH - envelopes plus tight
        "ma": [5, 7],                    # MA standard
        "size": [0.10, 0.12]             # Size conservateur
    },
    "mid-cap": {
        "mult": [1.0, 1.1, 1.2],        # SOL/AVAX - envelopes standard+
        "ma": [5, 7, 10],                # MA variable
        "size": [0.10, 0.12, 0.14]       # Size variable
    },
    "volatile": {
        "mult": [1.2, 1.3, 1.4],        # DOGE/SUSHI - envelopes larges
        "ma": [5, 7],                    # MA court pour réactivité
        "size": [0.12, 0.14]             # Size plus agressif
    },
    "low": {
        "mult": [1.0],                  # TRX - envelopes standard
        "ma": [7, 10],                   # MA long (peu de signaux)
        "size": [0.10]                   # Size conservateur
    }
}

# Fonction pour générer grilles par profil
def generate_profile_grid(profile, pairs_in_profile):
    """
    Génère la grille de configs pour un profil donné

    Args:
        profile: Nom du profil (major, mid-cap, volatile, low)
        pairs_in_profile: Liste des pairs dans ce profil

    Returns:
        List de dicts avec configs à tester
    """
    configs = []

    multipliers = PROFILE_MULTIPLIERS[profile]["mult"]
    ma_windows = PROFILE_MULTIPLIERS[profile]["ma"]
    sizes = PROFILE_MULTIPLIERS[profile]["size"]

    for mult in multipliers:
        for ma in ma_windows:
            for size in sizes:
                # Générer config pour chaque pair du profil
                pair_configs = {}

                for pair in pairs_in_profile:
                    # Déterminer nb envelopes depuis mapping
                    if df_envelope_mapping is not None and pair in df_envelope_mapping.index:
                        n_env = df_envelope_mapping.loc[pair, 'n_envelopes']
                    else:
                        # Fallback : 3 env par défaut
                        n_env = 3

                    # Sélectionner base selon nb envelopes
                    base_env = BASE_CONFIG[f'envelopes_{n_env}']

                    # Appliquer multiplicateur
                    envelopes = [round(e * mult, 3) for e in base_env]

                    pair_configs[pair] = {
                        'ma_base_window': ma,
                        'envelopes': envelopes,
                        'size': size / 10  # Ajusté pour leverage 10x (comme multi_envelope.ipynb)
                    }

                configs.append({
                    'profile': profile,
                    'mult': mult,
                    'ma': ma,
                    'size': size,
                    'stop_loss': BASE_CONFIG['stop_loss'],
                    'pair_configs': pair_configs,
                    'adaptive': False  # Fixed params par défaut
                })

    return configs

# Mapping pair -> profil (depuis profiles_map.csv ou manuel)
PAIR_PROFILES = {
    "BTC/USDT:USDT": "major",
    "ETH/USDT:USDT": "major",
    "BNB/USDT:USDT": "mid-cap",
    "SOL/USDT:USDT": "mid-cap",
    "ADA/USDT:USDT": "mid-cap",
    "AVAX/USDT:USDT": "mid-cap",
    "AR/USDT:USDT": "mid-cap",
    "ATOM/USDT:USDT": "mid-cap",
    "DOGE/USDT:USDT": "volatile",
    "SUSHI/USDT:USDT": "volatile",
    "GALA/USDT:USDT": "volatile",
    "TRX/USDT:USDT": "low",
}

# Charger depuis profiles_map.csv si disponible
if os.path.exists('profiles_map.csv'):
    df_profiles_map = pd.read_csv('profiles_map.csv')
    PAIR_PROFILES.update(dict(zip(df_profiles_map['pair'], df_profiles_map['profile'])))
    print(f"✅ Profiles map chargé : {len(PAIR_PROFILES)} pairs")

# Générer toutes les grilles par profil
PARAM_GRIDS_BY_PROFILE = {}

for profile in PROFILE_MULTIPLIERS.keys():
    # Filtrer pairs du profil
    pairs_in_profile = [pair for pair in PAIRS if PAIR_PROFILES.get(pair) == profile]

    if len(pairs_in_profile) > 0:
        grid = generate_profile_grid(profile, pairs_in_profile)
        PARAM_GRIDS_BY_PROFILE[profile] = grid

        print(f"Profil {profile:10s} : {len(grid):3d} configs × {len(pairs_in_profile)} pairs")

# Compter total configs
total_configs = sum(len(grid) for grid in PARAM_GRIDS_BY_PROFILE.values())
print(f"\n✅ Total configs profils : {total_configs}")

if not TEST_MODE:
    total_backtests = total_configs * len(WF_FOLDS) * 2  # Fixed + Adaptive
else:
    total_backtests = total_configs * len(WF_FOLDS)  # Fixed only en TEST_MODE

print(f"   Total backtests : {total_backtests} (Fixed + Adaptive)")

✅ Profiles map chargé : 30 pairs
Profil major      :  12 configs × 2 pairs
Profil mid-cap    :  27 configs × 3 pairs
Profil volatile   :  12 configs × 2 pairs
Profil low        :   2 configs × 1 pairs

✅ Total configs profils : 53
   Total backtests : 371 (Fixed + Adaptive)


In [6]:
# Chargement des données
exchange = ExchangeDataManager(
    exchange_name=EXCHANGE,
    path_download="../database/exchanges"  # Pointe vers Backtest-Tools-V2/database/exchanges
)

# Charger TOUTES les données nécessaires (2020-2025 - couvre tous les cycles)
start_date = "2020-01-01"
end_date = "2025-10-03"

df_list_full = {}
print("📥 Chargement des données...")
for pair in tqdm(PAIRS, desc="Paires"):
    df = exchange.load_data(pair, "1h", start_date=start_date, end_date=end_date)
    df_list_full[pair] = df

# BTC pour détection de régime
df_btc_full = exchange.load_data("BTC/USDT:USDT", "1h", start_date=start_date, end_date=end_date)

oldest_pair = min(df_list_full, key=lambda p: df_list_full[p].index.min())

print(f"\n✅ Données chargées")
print(f"   Période: {start_date} → {end_date} (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")
print(f"   Paire la plus ancienne: {oldest_pair}")
print(f"   Nombre de barres: {len(df_list_full[oldest_pair])}")

📥 Chargement des données...


Paires: 100%|██████████| 8/8 [00:00<00:00, 37.95it/s]


✅ Données chargées
   Période: 2020-01-01 → 2025-10-03 (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)
   Paire la plus ancienne: BTC/USDT:USDT
   Nombre de barres: 50458


## 2️⃣ Comparaison manuelle de configurations pré-définies

Teste 5 configurations fixes pour comprendre l'impact des paramètres.

In [ ]:
print("⚠️  Section 2️⃣ Comparaison manuelle SKIPPÉE (nouveau format)")

📋 5 configurations manuelles définies
   - Conservative: MA=10, Env=[0.05, 0.08, 0.12], Size=0.06
   - Standard (Live actuel): MA=7, Env=[0.07, 0.1, 0.15], Size=0.1
   - Aggressive: MA=5, Env=[0.09, 0.13, 0.18], Size=0.12
   - Wide Envelopes: MA=7, Env=[0.1, 0.15, 0.2], Size=0.08
   - Tight Envelopes: MA=7, Env=[0.05, 0.07, 0.1], Size=0.08


In [8]:
# Fonction helper pour run un backtest
def run_single_backtest(df_list, oldest_pair, params_coin, stop_loss, params_adapter=None):
    """
    Exécute un backtest avec les paramètres donnés.
    
    Returns:
        dict: Résultat du backtest (trades, days, wallet, metrics)
    """
    strategy = EnvelopeMulti_v2(
        df_list=df_list,
        oldest_pair=oldest_pair,
        type=["long", "short"],
        params=params_coin
    )
    
    strategy.populate_indicators()
    strategy.populate_buy_sell()
    
    result = strategy.run_backtest(
        **BACKTEST_PARAMS,
        stop_loss=stop_loss,
        params_adapter=params_adapter
    )
    
    return result

print("✅ Fonction run_single_backtest définie")

✅ Fonction run_single_backtest définie


In [9]:
# ======================
# PARALLÉLISATION MULTI-CORE
# ======================

def run_backtest_worker(args):
    """
    Worker function pour exécution parallèle d'un backtest.
    Doit être une fonction top-level pour être pickable par ProcessPoolExecutor.
    
    Args:
        args: tuple (config_dict, df_list, params_coin, stop_loss, adapter_params, is_adaptive)
    
    Returns:
        dict: Résultat du backtest avec métadonnées
    """
    config_dict, df_list_dict, params_coin, stop_loss, adapter_params, is_adaptive = args
    
    # Reconstituer les DataFrames depuis les dicts
    df_list = {pair: pd.DataFrame(data) for pair, data in df_list_dict.items()}
    
    # Trouver oldest_pair
    oldest_pair = min(df_list, key=lambda p: df_list[p].index.min())
    
    # Créer l'adapter
    if is_adaptive:
        regime_series = pd.Series(adapter_params['regime_data'], 
                                  index=pd.DatetimeIndex(adapter_params['regime_index']))
        adapter = RegimeBasedAdapter(
            base_params=params_coin,
            regime_series=regime_series,
            regime_params=DEFAULT_PARAMS,
            multipliers=adapter_params['multipliers'],
            base_std=adapter_params['base_std']
        )
    else:
        adapter = FixedParamsAdapter(params_coin)
    
    # Exécuter backtest
    strategy = EnvelopeMulti_v2(
        df_list=df_list,
        oldest_pair=oldest_pair,
        type=["long", "short"],
        params=params_coin
    )
    
    strategy.populate_indicators()
    strategy.populate_buy_sell()
    
    result = strategy.run_backtest(
        **BACKTEST_PARAMS,
        stop_loss=stop_loss,
        params_adapter=adapter
    )
    
    return {
        'config': config_dict,
        'wallet': result['days']['wallet'].iloc[-1] if len(result['days']) > 0 else INITIAL_WALLET,
        'sharpe': result.get('sharpe_ratio', 0),
        'n_trades': len(result['trades']),
        'result': result  # Garder le résultat complet
    }


def run_backtests_parallel(configs, df_list, regime_series=None, max_workers=None):
    """
    Exécute plusieurs backtests en parallèle sur plusieurs cores CPU.
    
    Args:
        configs: list de dicts avec les paramètres de config
        df_list: dict de DataFrames par paire
        regime_series: Series des régimes (si adaptive)
        max_workers: nombre de workers (None = auto)
    
    Returns:
        list: Résultats des backtests
    """
    # Préparer arguments pour chaque worker
    tasks = []
    
    for config in configs:
        # Convertir DataFrames en dicts pour serialization
        df_list_dict = {pair: df.to_dict('list') for pair, df in df_list.items()}
        
        # Préparer params_coin
        params_coin = {}
        for pair in df_list.keys():
            params_coin[pair] = {
                "src": "close",
                "ma_base_window": config['ma_window'],
                "envelopes": config['envelopes'],
                "size": config['size'] / BACKTEST_LEVERAGE
            }
        
        # Adapter params
        if config['adaptive']:
            adapter_params = {
                'regime_data': regime_series.values.tolist(),
                'regime_index': regime_series.index.tolist(),
                'multipliers': {'envelope_std': True},
                'base_std': 0.10
            }
        else:
            adapter_params = None
        
        tasks.append((
            config,
            df_list_dict,
            params_coin,
            config['stop_loss'],
            adapter_params,
            config['adaptive']
        ))
    
    # Exécution parallèle
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(run_backtest_worker, task): i for i, task in enumerate(tasks)}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Backtests parallèles"):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"Erreur backtest: {e}")
                results.append(None)
    
    return results


print("✅ Fonctions de parallélisation définies")
print(f"   Gain attendu: ~4-5x avec votre i9-14900HX")

✅ Fonctions de parallélisation définies
   Gain attendu: ~4-5x avec votre i9-14900HX


In [10]:
# Calculer les régimes sur TOUTE la période (pour comparaison manuelle seulement)
# ⚠️ Pour Walk-Forward, on recalculera par fold
regime_series_full = calculate_regime_series(df_btc_full, confirm_n=12)

print("📊 Distribution des régimes (2020-2025):")
regime_counts = regime_series_full.value_counts(normalize=True) * 100
for regime, pct in regime_counts.items():
    print(f"   {regime.name}: {pct:.1f}%")

📊 Distribution des régimes (2020-2025):
   BULL: 48.3%
   BEAR: 39.1%
   RECOVERY: 12.6%


In [25]:
# ========================================
# Section 2️⃣ : Comparaison Manuelle SKIP
# ========================================
print("⚠️  Section comparaison manuelle SKIPPÉE (nouveau format grilles)")
print("   Passer directement à Walk-Forward Optimization\n")

# Si tu veux vraiment tester une config manuellement :
# 1. Choisis un profil
# 2. Prends la première config de PARAM_GRIDS_BY_PROFILE[profile]
# 3. Lance run_single_backtest avec cette config

# Exemple (optionnel) :
if False:  # Mettre True pour activer
    test_profile = "major"
    test_config = PARAM_GRIDS_BY_PROFILE[test_profile][0]
    
    pairs_in_profile = [pair for pair in PAIRS if PAIR_PROFILES.get(pair) == test_profile]
    df_list_profile = {pair: df_list_full[pair] for pair in pairs_in_profile if pair in df_list_full}
    
    params_coin = test_config['pair_configs']
    
    adapter_fixed = FixedParamsAdapter(params_coin)
    result = run_single_backtest(
        df_list_profile,
        oldest_pair,
        params_coin,
        test_config['stop_loss'],
        adapter_fixed
    )
    
    print(f"Test config {test_profile}: Sharpe={result.get('sharpe_ratio', 0):.2f}")

⚠️  Section comparaison manuelle SKIPPÉE (nouveau format grilles)
   Passer directement à Walk-Forward Optimization



In [26]:
# Afficher les résultats
comparator_manual.print_summary()


🔍 COMPARAISON DES BACKTESTS
            Strategy  Final Wallet  Total Perf (%)  Sharpe Ratio  Max DD (%)  Win Rate (%)  N Trades  Avg PnL (%)  Max Win (%)  Max Loss (%)  Total Fees  Avg Exposition  Avg Long Expo  Avg Short Expo  Avg Duration (h)
Conservative (Fixed)   1388.577859       38.857786       2.03501   -2.213967     72.709163       502      1.32378    11.024478     -9.753471    11.77701        2.334245       1.413402        0.920843          3.782869

--------------------------------------------------------------------------------
✅ RECOMMANDATION: Conservative (Fixed)
--------------------------------------------------------------------------------



In [27]:
# Sauvegarder les résultats manuels
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
comparator_manual.save_comparison(f"results_manual_{timestamp}.csv")
print(f"💾 Résultats sauvegardés: results_manual_{timestamp}.csv")

✅ Comparaison sauvegardée: results_manual_20251005_105352.csv
💾 Résultats sauvegardés: results_manual_20251005_105352.csv


In [28]:
# Grid de paramètres (INTERMÉDIAIRE - optimisation globale)
# Étape 1: Valider avec grid élargi avant d'implémenter optimisation par profil
PARAM_GRID = {
    "ma_base_window": [5, 7, 10],           # 3 valeurs (réactivité)
    "envelope_sets": [
        [0.07, 0.10, 0.15],                 # Standard (actuel live)
        [0.10, 0.15, 0.20],                 # Wide (pour volatiles)
    ],                                       # 2 sets
    "size": [0.08, 0.10, 0.12],             # 3 valeurs (risque)
    "stop_loss": [0.25],                    # 1 valeur (garder simple)
}

# Générer toutes les combinaisons
grid_combinations = list(product(
    PARAM_GRID["ma_base_window"],
    PARAM_GRID["envelope_sets"],
    PARAM_GRID["size"],
    PARAM_GRID["stop_loss"]
))

print(f"🔍 Grid Search (INTERMÉDIAIRE - optimisation globale)")
print(f"   Combinaisons: {len(grid_combinations)}")
print(f"   Walk-Forward Folds: {len(WF_FOLDS)}")
print(f"   Total backtests: {len(grid_combinations) * len(WF_FOLDS) * 2} (fixed + adaptive)")
print(f"   Temps estimé: ~{len(grid_combinations) * len(WF_FOLDS) * 2 * 3 / 60:.0f} min avec multi-core")
print(f"   Période: 2020-2025 (couvre BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024-25)")
print(f"\n💡 Approche incrémentale:")
print(f"   Étape 1 (actuelle): Grid intermédiaire global (18 configs)")
print(f"   Étape 2 (si besoin): Optimisation par profil (4 grids séparés)")

🔍 Grid Search (INTERMÉDIAIRE - optimisation globale)
   Combinaisons: 18
   Walk-Forward Folds: 7
   Total backtests: 252 (fixed + adaptive)
   Temps estimé: ~13 min avec multi-core
   Période: 2020-2025 (couvre BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024-25)

💡 Approche incrémentale:
   Étape 1 (actuelle): Grid intermédiaire global (18 configs)
   Étape 2 (si besoin): Optimisation par profil (4 grids séparés)


In [30]:
# Fonction pour filtrer DataFrame par dates
def filter_df_by_dates(df, start_date, end_date):
    """Filtre un DataFrame par dates."""
    mask = (df.index >= pd.Timestamp(start_date)) & (df.index <= pd.Timestamp(end_date))
    return df[mask]

def filter_df_list_by_dates(df_list, start_date, end_date):
    """Filtre un dict de DataFrames par dates."""
    return {pair: filter_df_by_dates(df, start_date, end_date) for pair, df in df_list.items()}

print("✅ Fonctions de filtrage définies")

✅ Fonctions de filtrage définies


In [31]:
# Fonction de calcul du score composite anti-overfitting
def calculate_composite_score(bt_result, train_sharpe=None):
    """
    Calcule le score composite pour évaluer une configuration.
    
    Args:
        bt_result: Résultat du backtest
        train_sharpe: Sharpe du train (pour consistency), None si on calcule train
    
    Returns:
        float: Score composite (plus élevé = meilleur)
    """
    df_trades = bt_result['trades']
    df_days = bt_result['days']
    
    # Calculer métriques de base
    n_trades = len(df_trades)
    
    # Filtre: trop peu de trades = pas fiable
    if n_trades < 10:
        return -999  # Sera skip par early termination de toute façon
    
    # Poids réduit pour échantillons 10-30 trades
    if n_trades < 30:
        weight_penalty = n_trades / 30  # 0.33 à 1.0
    else:
        weight_penalty = 1.0
    
    # Sharpe ratio
    sharpe = bt_result.get('sharpe_ratio', 0)
    if pd.isna(sharpe) or np.isinf(sharpe):
        sharpe = 0
    
    # Clip sharpe pour éviter outliers
    sharpe = np.clip(sharpe, -5, 10)
    
    # Max Drawdown
    df_days_copy = df_days.copy()
    df_days_copy['cummax'] = df_days_copy['wallet'].cummax()
    df_days_copy['drawdown_pct'] = (df_days_copy['wallet'] - df_days_copy['cummax']) / df_days_copy['cummax']
    max_dd = abs(df_days_copy['drawdown_pct'].min()) * 100
    
    # Calmar Ratio (return / max_dd)
    final_wallet = df_days['wallet'].iloc[-1]
    total_return = (final_wallet / INITIAL_WALLET - 1) * 100
    calmar = total_return / max(max_dd, 1.0)  # Éviter division par 0
    calmar = np.clip(calmar, -5, 10)  # Clip calmar
    
    # Win Rate
    df_trades_copy = df_trades.copy()
    if 'trade_result' not in df_trades_copy.columns:
        df_trades_copy['trade_result'] = (
            df_trades_copy["close_trade_size"] -
            df_trades_copy["open_trade_size"] -
            df_trades_copy["open_fee"] -
            df_trades_copy["close_fee"]
        )
    win_rate = (df_trades_copy['trade_result'] > 0).mean()
    
    # Profit Factor
    gross_profit = df_trades_copy[df_trades_copy['trade_result'] > 0]['trade_result'].sum()
    gross_loss = abs(df_trades_copy[df_trades_copy['trade_result'] < 0]['trade_result'].sum())
    profit_factor = gross_profit / max(gross_loss, 1.0) if gross_loss > 0 else gross_profit
    profit_factor_normalized = np.clip(profit_factor / 2, 0, 1)  # Cap à 1
    
    # Consistency (train vs test)
    if train_sharpe is not None:
        consistency = 1 - abs(train_sharpe - sharpe) / max(0.1, abs(train_sharpe))
        consistency = np.clip(consistency, 0, 1)  # Clip à [0,1]
    else:
        consistency = 0  # Pas de consistency pour train
    
    # DD factor
    dd_factor = np.clip(1 - min(max_dd, 100) / 100, 0, 1)
    
    # Score composite
    if train_sharpe is None:  # Train
        score = (
            sharpe * 0.35 +
            calmar * 0.25 +
            dd_factor * 0.20 +
            win_rate * 0.10 +
            profit_factor_normalized * 0.10
        ) * weight_penalty
    else:  # Test
        score = (
            sharpe * 0.30 +
            consistency * 0.25 +
            calmar * 0.20 +
            dd_factor * 0.15 +
            win_rate * 0.05 +
            profit_factor_normalized * 0.05
        ) * weight_penalty
    
    return score

print("✅ Fonction calculate_composite_score définie")

✅ Fonction calculate_composite_score définie


In [32]:
# Cache désactivé temporairement (nouveau format de grilles)
# Le cache sera réactivé après adaptation à PARAM_GRIDS_BY_PROFILE
cache = None
print("⚠️  Cache indicateurs désactivé (nouveau format grilles)")

⚠️  Cache indicateurs désactivé (nouveau format grilles)


In [ ]:
# =================================================================
# CELL-19 OPTIMISÉE - Walk-Forward avec Palier 1 (×1.5-2.5 gain)
# =================================================================
# Optimisations:
# 1. Cache des indicateurs (pré-calcul)
# 2. Early termination (skip configs non-viables)
# 3. Batching intelligent (réduction overhead)

from indicator_cache import IndicatorCache, precompute_all_indicators

# Initialiser le cache
cache = IndicatorCache(cache_dir="./cache_indicators")

# Cache désactivé temporairement (nouveau format de grilles incompatible)
# precompute_all_indicators(df_list_full, PARAM_GRIDS_BY_PROFILE, PERIODS, cache)
cache = None
print("⚠️  Cache indicateurs désactivé (nouveau format grilles)")

# Walk-Forward Optimization PAR PROFIL (OPTIMISÉE)
wf_results_by_profile = {}

print("\n🚀 Démarrage Walk-Forward Optimization PAR PROFIL (OPTIMISÉE)...\n")
print("=" * 80)

for profile in PARAM_GRIDS_BY_PROFILE.keys():
    print(f"\n{'=' * 80}")
    print(f"🔬 OPTIMISATION PROFIL: {profile.upper()}")
    print(f"{'=' * 80}")

    # Filtrer les paires du profil
    pairs_in_profile = [pair for pair in PAIRS if PAIR_PROFILES.get(pair) == profile]

    if len(pairs_in_profile) == 0:
        print(f"⚠️  Aucune paire dans le profil {profile}, skip")
        continue

    print(f"   Paires: {', '.join(pairs_in_profile)} ({len(pairs_in_profile)} paires)")

    # Générer combinaisons pour ce profil
    grid = PARAM_GRIDS_BY_PROFILE[profile]
    # grid est maintenant une liste de configs (nouveau format)
    grid_combinations_profile = grid  # Pas besoin de product(), les configs sont déjà générées

    print(f"   Configs à tester: {len(grid_combinations_profile)}")

    # Calculer iterations selon TEST_MODE
    if TEST_MODE:
        total_iterations = len(WF_FOLDS) * len(grid_combinations_profile)
        print(f"   🧪 MODE TEST: Fixed only (skip Adaptive)")
    else:
        total_iterations = len(WF_FOLDS) * len(grid_combinations_profile) * 2

    print(f"   Total backtests: {total_iterations}")

    # Walk-Forward Loop avec early termination
    wf_results = []
    skipped_configs = 0
    pbar = tqdm(total=total_iterations, desc=f"{profile.upper()} WFO")

    for combo_idx, config in enumerate(grid_combinations_profile):
        # Extraire params de la config
        ma = config['ma']
        size = config['size']
        sl = config['stop_loss']
        pair_configs = config['pair_configs']  # Nouveau : configs par pair

        # Early termination: skip si config déjà mauvaise sur premiers folds
        should_skip = False
        fold_count = 0

        for fold in WF_FOLDS:
            fold_name = fold["name"]
            fold_count += 1

            # Filtrer données par période
            df_list_train = filter_df_list_by_dates(df_list_full, fold['train_start'], fold['train_end'])
            df_list_test = filter_df_list_by_dates(df_list_full, fold['test_start'], fold['test_end'])

            df_btc_train = filter_df_by_dates(df_btc_full, fold['train_start'], fold['train_end'])
            df_btc_test = filter_df_by_dates(df_btc_full, fold['test_start'], fold['test_end'])

            # Filtrer par profil
            df_list_train_profile = {p: df for p, df in df_list_train.items() if p in pairs_in_profile}
            df_list_test_profile = {p: df for p, df in df_list_test.items() if p in pairs_in_profile}

            # Calculer régimes par fold
            regime_train = calculate_regime_series(df_btc_train, confirm_n=12)
            regime_test = calculate_regime_series(df_btc_test, confirm_n=12)

            # Garde-fou : Vérifier fold valide
            if len(df_list_train_profile) == 0 or len(df_list_test_profile) == 0:
                print(f"      ⚠️  {fold_name}: Données insuffisantes, skip fold")
                pbar.update(1 if TEST_MODE else 2)
                continue

            # Préparer params_coin depuis pair_configs (déjà dans le bon format)
            params_coin = {}
            for pair in pairs_in_profile:
                if pair in pair_configs:
                    params_coin[pair] = {
                        "src": "close",
                        **pair_configs[pair]  # Contient déjà ma_base_window, envelopes, size
                    }

            # === TRAIN ===
            adapter_fixed = FixedParamsAdapter(params_coin)
            bt_train_fixed = run_single_backtest(
                df_list_train_profile, min(df_list_train_profile, key=lambda p: df_list_train_profile[p].index.min()),
                params_coin, sl, adapter_fixed
            )
            score_train_fixed = calculate_composite_score(bt_train_fixed)
            sharpe_train_fixed = bt_train_fixed.get('sharpe_ratio', 0)

            # === TEST ===
            adapter_fixed_test = FixedParamsAdapter(params_coin)
            bt_test_fixed = run_single_backtest(
                df_list_test_profile, min(df_list_test_profile, key=lambda p: df_list_test_profile[p].index.min()),
                params_coin, sl, adapter_fixed_test
            )
            score_test_fixed = calculate_composite_score(bt_test_fixed, sharpe_train_fixed)

            # 🚀 EARLY TERMINATION : Skip si trop peu de trades ou DD élevé sur les 2 premiers folds
            if fold_count <= 2:  # Évaluer sur les 2 premiers folds
                n_trades = len(bt_test_fixed['trades'])

                # Calculer max DD
                df_days = bt_test_fixed['days']
                if len(df_days) > 0:
                    df_days_copy = df_days.copy()
                    df_days_copy['cummax'] = df_days_copy['wallet'].cummax()
                    df_days_copy['drawdown_pct'] = (df_days_copy['wallet'] - df_days_copy['cummax']) / df_days_copy['cummax']
                    max_dd = abs(df_days_copy['drawdown_pct'].min()) * 100
                else:
                    max_dd = 0

                # Conditions d'élimination précoce
                if n_trades < 10:  # Trop peu de trades
                    should_skip = True
                    skip_reason = f"<10 trades (fold {fold_count})"
                elif max_dd > 50:  # DD trop élevé
                    should_skip = True
                    skip_reason = f"DD>{max_dd:.1f}% (fold {fold_count})"
                elif score_test_fixed < -500:  # Score catastrophique
                    should_skip = True
                    skip_reason = f"score<-500 (fold {fold_count})"

            # Stocker résultats Fixed
            wf_results.append({
                "profile": profile,
                "fold": fold_name,
                "combo_idx": combo_idx,
                "ma_window": ma,
                "envelopes": str(list(pair_configs.values())[0]['envelopes']) if pair_configs else "[]",
                "size": size,
                "stop_loss": sl,
                "adaptive": False,
                "train_wallet": bt_train_fixed['wallet'],
                "train_sharpe": sharpe_train_fixed,
                "train_score": score_train_fixed,
                "train_trades": len(bt_train_fixed['trades']),
                "test_wallet": bt_test_fixed['wallet'],
                "test_sharpe": bt_test_fixed.get('sharpe_ratio', 0),
                "test_score": score_test_fixed,
                "test_trades": len(bt_test_fixed['trades']),
            })
            pbar.update(1)

            # === ADAPTIVE (skip en mode TEST) ===
            if not TEST_MODE:
                adapter_adaptive_train = RegimeBasedAdapter(
                    base_params=params_coin,
                    regime_series=regime_train,
                    regime_params=DEFAULT_PARAMS,
                    multipliers={'envelope_std': True},
                    base_std=0.10
                )
                bt_train_adaptive = run_single_backtest(
                    df_list_train_profile, min(df_list_train_profile, key=lambda p: df_list_train_profile[p].index.min()),
                    params_coin, stop_loss, adapter_adaptive_train
                )
                score_train_adaptive = calculate_composite_score(bt_train_adaptive)
                sharpe_train_adaptive = bt_train_adaptive.get('sharpe_ratio', 0)

                adapter_adaptive_test = RegimeBasedAdapter(
                    base_params=params_coin,
                    regime_series=regime_test,
                    regime_params=DEFAULT_PARAMS,
                    multipliers={'envelope_std': True},
                    base_std=0.10
                )
                bt_test_adaptive = run_single_backtest(
                    df_list_test_profile, min(df_list_test_profile, key=lambda p: df_list_test_profile[p].index.min()),
                    params_coin, stop_loss, adapter_adaptive_test
                )
                score_test_adaptive = calculate_composite_score(bt_test_adaptive, sharpe_train_adaptive)

                wf_results.append({
                    "profile": profile,
                    "fold": fold_name,
                    "combo_idx": combo_idx,
                    "ma_window": ma,
                    "envelopes": str(list(pair_configs.values())[0]['envelopes']) if pair_configs else "[]",
                    "size": size,
                    "stop_loss": stop_loss,
                    "adaptive": True,
                    "train_wallet": bt_train_adaptive['wallet'],
                    "train_sharpe": sharpe_train_adaptive,
                    "train_score": score_train_adaptive,
                    "train_trades": len(bt_train_adaptive['trades']),
                    "test_wallet": bt_test_adaptive['wallet'],
                    "test_sharpe": bt_test_adaptive.get('sharpe_ratio', 0),
                    "test_score": score_test_adaptive,
                    "test_trades": len(bt_test_adaptive['trades']),
                })
                pbar.update(1)

            # Si early termination détectée, skip les folds restants
            if should_skip:
                remaining_folds = len(WF_FOLDS) - fold_count
                pbar.update(remaining_folds * (1 if TEST_MODE else 2))
                skipped_configs += 1
                print(f"      ⏭️  Config#{combo_idx+1} skipped: {skip_reason}")
                break  # Sort de la boucle des folds

    pbar.close()
    wf_results_by_profile[profile] = pd.DataFrame(wf_results)
    print(f"   ✅ {len(wf_results)} résultats enregistrés pour {profile}")
    if skipped_configs > 0:
        print(f"   ⏭️  {skipped_configs} configs skipped (early termination)")

print("\n" + "=" * 80)
print("✅ Walk-Forward Optimization PAR PROFIL terminée (OPTIMISÉE)\n")


⚠️  Cache indicateurs désactivé (nouveau format grilles)

🚀 Démarrage Walk-Forward Optimization PAR PROFIL (OPTIMISÉE)...


🔬 OPTIMISATION PROFIL: MAJOR
   Paires: BTC/USDT:USDT, ETH/USDT:USDT (2 paires)
   Configs à tester: 12
   🧪 MODE TEST: Fixed only (skip Adaptive)
   Total backtests: 84


MAJOR WFO: 100%|██████████| 84/84 [04:48<00:00,  3.43s/it]


   ✅ 84 résultats enregistrés pour major

🔬 OPTIMISATION PROFIL: MID-CAP
   Paires: SOL/USDT:USDT, AVAX/USDT:USDT, ADA/USDT:USDT (3 paires)
   Configs à tester: 27
   🧪 MODE TEST: Fixed only (skip Adaptive)
   Total backtests: 189


MID-CAP WFO: 100%|██████████| 189/189 [17:18<00:00,  5.50s/it] 


   ✅ 189 résultats enregistrés pour mid-cap

🔬 OPTIMISATION PROFIL: VOLATILE
   Paires: DOGE/USDT:USDT, SUSHI/USDT:USDT (2 paires)
   Configs à tester: 12
   🧪 MODE TEST: Fixed only (skip Adaptive)
   Total backtests: 84


VOLATILE WFO:  64%|██████▍   | 54/84 [03:01<01:41,  3.37s/it]

In [ ]:
# Combiner les résultats de tous les profils
df_wf_all_profiles = pd.concat(list(wf_results_by_profile.values()), ignore_index=True)

print("✅ Résultats combinés de tous les profils")
print(f"   Total résultats: {len(df_wf_all_profiles)}")
print(f"   Profils: {df_wf_all_profiles['profile'].unique().tolist()}")

# Meilleure config PAR PROFIL
best_configs_by_profile = {}

# Garde-fou #5 : Filtre trades minimum par profil
MIN_TRADES_PER_PROFILE = 50

# Pas de hard cutoff MIN_TRADES - utiliser système de poids à la place
for profile in PARAM_GRIDS_BY_PROFILE.keys():
    df_profile = df_wf_all_profiles[df_wf_all_profiles['profile'] == profile]
    
    if len(df_profile) == 0:
        print(f"\n⚠️  Profil {profile}: Aucun résultat (profil skip)")
        continue

    # Agréger par config
    df_profile_avg = df_profile.groupby(['ma_window', 'envelopes', 'size', 'stop_loss', 'adaptive']).agg({
        'train_score': 'mean',
        'test_score': 'mean',
        'train_sharpe': 'mean',
        'test_sharpe': 'mean',
        'train_trades': 'sum',
        'test_trades': 'sum',
    }).reset_index()
    
    # Calculer consistency
    df_profile_avg['consistency'] = 1 - abs(df_profile_avg['train_sharpe'] - df_profile_avg['test_sharpe']) / df_profile_avg['train_sharpe'].abs().clip(lower=0.1)
    df_profile_avg['consistency'] = df_profile_avg['consistency'].clip(lower=0)
    
    # Ajouter poids si pas présent (configs avec peu de trades ont déjà weight réduit dans scoring)
    if 'weight' not in df_profile_avg.columns:
        # Calculer poids basé sur nombre de trades
        df_profile_avg['weight'] = df_profile_avg['test_trades'].apply(
            lambda x: 1.0 if x >= 50 else (0.25 if x >= 30 else (x / 30 if x >= 10 else 0.1))
        )
    
    # Score pondéré
    df_profile_avg['weighted_score'] = df_profile_avg['test_score'] * df_profile_avg['weight']
    
    # Trier par score pondéré (pas de filtrage dur)
    df_profile_avg = df_profile_avg.sort_values('weighted_score', ascending=False)
    
    # Prendre le meilleur (pas de MIN_TRADES cutoff)
    if len(df_profile_avg) > 0:
        best_configs_by_profile[profile] = df_profile_avg.iloc[0]
        
        # Warning si peu de trades
        n_trades = df_profile_avg.iloc[0]['test_trades']
        weight = df_profile_avg.iloc[0]['weight']
        if n_trades < 50:
            print(f"\n⚠️  Profil {profile}: {n_trades} trades (weight={weight:.2f})")
        else:
            print(f"\n✅ Profil {profile}: {n_trades} trades")
    else:
        print(f"\n❌ Profil {profile}: Aucun résultat")

print(f"\n{'=' * 80}")
print("🏆 MEILLEURES CONFIGURATIONS PAR PROFIL")
print(f"{'=' * 80}\n")

for profile, best_cfg in best_configs_by_profile.items():
    print(f"{profile.upper()}:")
    print(f"   MA: {int(best_cfg['ma_window'])}, Env: {best_cfg['envelopes']}, Size: {best_cfg['size']:.2f}, SL: {best_cfg['stop_loss']}")
    print(f"   Adaptive: {best_cfg['adaptive']}")
    print(f"   Train Sharpe: {best_cfg['train_sharpe']:.2f}, Test Sharpe: {best_cfg['test_sharpe']:.2f}")
    print(f"   Test Score: {best_cfg['test_score']:.3f}, Consistency: {best_cfg['consistency']:.2f}")
    print(f"   Trades: {int(best_cfg['test_trades'])}, Weight: {best_cfg.get('weight', 1.0):.2f}\n")

In [ ]:
# ======================
# GATE : Profil vs Global
# ======================

# Charger résultats globaux (Étape 1)
try:
    df_wf_global = pd.read_csv('wf_results_summary_20251004_235003.csv')  # Résultat Étape 1
    
    # Score global moyen (meilleur)
    best_global_score = df_wf_global['test_score'].max()
    best_global_sharpe = df_wf_global.loc[df_wf_global['test_score'].idxmax(), 'test_sharpe']
    best_global_config = df_wf_global.loc[df_wf_global['test_score'].idxmax()]
    
    print(f"\n{'=' * 80}")
    print("📊 GATE : Optimisation Profil vs Optimisation Globale")
    print(f"{'=' * 80}\n")
    
    print("🔵 OPTIMISATION GLOBALE (Étape 1)")
    print(f"   MA: {best_global_config['ma_window']}, Env: {best_global_config['envelopes']}, Size: {best_global_config['size']}")
    print(f"   Test Score: {best_global_score:.3f}")
    print(f"   Test Sharpe: {best_global_sharpe:.2f}\n")
    
    # Score profil moyen (moyenne pondérée par nombre de paires)
    profile_scores = []
    for profile, best_cfg in best_configs_by_profile.items():
        # Compter paires dans échantillon
        n_pairs = len([p for p in PAIRS if PAIR_PROFILES.get(p) == profile])
        profile_scores.append({
            'profile': profile,
            'score': best_cfg['test_score'],
            'sharpe': best_cfg['test_sharpe'],
            'weight': n_pairs
        })
    
    df_profile_scores = pd.DataFrame(profile_scores)
    weighted_avg_score = (df_profile_scores['score'] * df_profile_scores['weight']).sum() / df_profile_scores['weight'].sum()
    weighted_avg_sharpe = (df_profile_scores['sharpe'] * df_profile_scores['weight']).sum() / df_profile_scores['weight'].sum()
    
    print("🟢 OPTIMISATION PAR PROFIL (Étape 2)")
    print(f"   Weighted Avg Score: {weighted_avg_score:.3f}")
    print(f"   Weighted Avg Sharpe: {weighted_avg_sharpe:.2f}\n")
    
    print(f"{'=' * 80}")
    print(f"Δ Score:  {weighted_avg_score - best_global_score:+.3f}")
    print(f"Δ Sharpe: {weighted_avg_sharpe - best_global_sharpe:+.2f}")
    print(f"{'=' * 80}\n")
    
    # Garde-fou #6 : Décision
    if weighted_avg_score > best_global_score and abs(weighted_avg_sharpe - best_global_sharpe) <= 0.5:
        print("✅ GATE PASSÉ: Optimisation par profil améliore les résultats")
        print("   → Recommandation: Adopter configs par profil\n")
        RECOMMENDATION = "profil"
    elif weighted_avg_score > best_global_score * 0.95:  # Tolérance 5%
        print("⚠️  GATE PARTIEL: Optimisation par profil légèrement meilleure")
        print("   → Recommandation: Évaluer sur 28 paires avant décision finale\n")
        RECOMMENDATION = "profil_conditional"
    else:
        print("❌ GATE ÉCHOUÉ: Optimisation globale reste meilleure")
        print("   → Recommandation: Garder config globale unique\n")
        RECOMMENDATION = "global"
    
except FileNotFoundError:
    print("⚠️  Résultats Étape 1 (global) non trouvés")
    print("   Gate skip, adopter configs par profil par défaut\n")
    RECOMMENDATION = "profil"
    
print(f"{'=' * 80}")
print(f"🎯 RECOMMANDATION FINALE: {RECOMMENDATION.upper()}")
print(f"{'=' * 80}")

In [ ]:
# Top 10 configurations
print("\n🏆 TOP 10 CONFIGURATIONS (par Test Score moyen)\n")
print("=" * 120)

top10 = df_wf_avg.head(10)
for idx, row in top10.iterrows():
    print(f"#{idx+1}")
    print(f"   MA: {row['ma_window']}, Env: {row['envelopes']}, Size: {row['size']}, SL: {row['stop_loss']}, Adaptive: {row['adaptive']}")
    print(f"   Train Sharpe: {row['train_sharpe']:.2f}, Test Sharpe: {row['test_sharpe']:.2f}, Consistency: {row['consistency']:.2f}")
    print(f"   Train Score: {row['train_score']:.3f}, Test Score: {row['test_score']:.3f}")
    print(f"   Trades: Train={row['train_trades']}, Test={row['test_trades']}")
    print()

# Identifier le meilleur
best_config = top10.iloc[0]
print("\n" + "=" * 120)
print(f"✅ MEILLEURE CONFIGURATION:")
print(f"   MA: {best_config['ma_window']}")
print(f"   Envelopes: {best_config['envelopes']}")
print(f"   Size: {best_config['size']}")
print(f"   Stop Loss: {best_config['stop_loss']}")
print(f"   Adaptive: {best_config['adaptive']}")
print(f"   Test Score: {best_config['test_score']:.3f}")

In [ ]:
# Comparaison Phase A (8 paires) vs Phase B (28 paires)
print("📊 COMPARAISON PHASE A vs PHASE B")
print("=" * 80)

# Limiter à top3 uniquement (car df_portfolio peut contenir plus de 3 configs)
for i in range(min(len(df_portfolio), len(top3))):
    row_portfolio = df_portfolio.iloc[i]
    row_phase_a = top3.iloc[i]
    
    print(f"\n{row_portfolio['config']}:")
    print(f"   MA={row_portfolio['ma_window']}, Env={row_portfolio['envelopes']}, Adaptive={row_portfolio['adaptive']}")
    print(f"   Phase A (8 paires):  Score={row_phase_a['test_score']:.3f}, Sharpe={row_phase_a['test_sharpe']:.2f}")
    print(f"   Phase B (28 paires): Score={row_portfolio['score']:.3f}, Sharpe={row_portfolio['sharpe']:.2f}")
    
    # Vérifier consistency
    score_diff = abs(row_portfolio['score'] - row_phase_a['test_score'])
    sharpe_diff = abs(row_portfolio['sharpe'] - row_phase_a['test_sharpe'])
    
    if sharpe_diff <= 0.5:
        print(f"   ✅ ROBUSTE: Sharpe diff = {sharpe_diff:.2f} (≤0.5)")
    else:
        print(f"   ⚠️  DIVERGENCE: Sharpe diff = {sharpe_diff:.2f} (>0.5)")

# Sélectionner la meilleure config pour hold-out
best_config_portfolio = df_portfolio.iloc[0]

print("\n" + "=" * 80)
print(f"🏆 MEILLEURE CONFIG POUR HOLD-OUT:")
print(f"   {best_config_portfolio['config']}")
print(f"   MA: {best_config_portfolio['ma_window']}")
print(f"   Envelopes: {best_config_portfolio['envelopes']}")
print(f"   Size: {best_config_portfolio['size']}")
print(f"   Stop Loss: {best_config_portfolio['stop_loss']}")
print(f"   Adaptive: {best_config_portfolio['adaptive']}")
print(f"   Score Portfolio: {best_config_portfolio['score']:.3f}")
print("=" * 80)

In [ ]:
# Tester top-3 configs sur 28 paires
top3 = df_wf_avg.head(3)

portfolio_results = []

for idx, row in top3.iterrows():
    config_name = f"Config#{idx+1}"
    print(f"\n🔄 Test {config_name}: MA={row['ma_window']}, Env={row['envelopes']}, Adaptive={row['adaptive']}")
    
    # Préparer params
    params_coin_28 = {}
    for pair in df_list_full_28.keys():
        params_coin_28[pair] = {
            "src": "close",
            "ma_base_window": int(row['ma_window']),
            "envelopes": eval(row['envelopes']),
            "size": float(row['size']) / BACKTEST_LEVERAGE
        }
    
    # Adapter
    if row['adaptive']:
        adapter = RegimeBasedAdapter(
            base_params=params_coin_28,
            regime_series=regime_series_full_28,
            regime_params=DEFAULT_PARAMS,
            multipliers={'envelope_std': True},
            base_std=0.10
        )
    else:
        adapter = FixedParamsAdapter(params_coin_28)
    
    # Run backtest
    result = run_single_backtest(
        df_list_full_28, oldest_pair_28, params_coin_28,
        float(row['stop_loss']), adapter
    )
    
    # Métriques
    final_wallet = result['wallet']
    sharpe = result.get('sharpe_ratio', 0)
    n_trades = len(result['trades'])
    
    # Score composite (comme Phase A)
    score = calculate_composite_score(result)
    
    portfolio_results.append({
        'config': config_name,
        'ma_window': int(row['ma_window']),
        'envelopes': row['envelopes'],
        'size': float(row['size']),
        'stop_loss': float(row['stop_loss']),
        'adaptive': bool(row['adaptive']),
        'wallet': final_wallet,
        'sharpe': sharpe,
        'n_trades': n_trades,
        'score': score,
    })
    
    print(f"   Wallet: ${final_wallet:.2f}, Sharpe: {sharpe:.2f}, Trades: {n_trades}, Score: {score:.3f}")

df_portfolio = pd.DataFrame(portfolio_results).sort_values('score', ascending=False)

print("\n" + "=" * 80)
print("✅ Validation portfolio complétée\n")

In [ ]:
# Charger données pour 28 paires
df_list_full_28 = {}
print("\n📥 Chargement des 28 paires...")
for pair in tqdm(PAIRS_FULL, desc="Paires"):
    try:
         # IMPORTANT: Charger jusqu'à la fin pour avoir les données hold-out
        df = exchange.load_data(pair, "1h", start_date="2020-01-01", end_date="2025-10-03")
        df_list_full_28[pair] = df
    except FileNotFoundError:
        print(f"⚠️  {pair} non disponible, ignoré")

oldest_pair_28 = min(df_list_full_28, key=lambda p: df_list_full_28[p].index.min())

# Régimes sur BTC (même période)
df_btc_full_28 = exchange.load_data("BTC/USDT:USDT", "1h", 
                                     start_date=PERIODS['train_full']['start'], 
                                     end_date=PERIODS['train_full']['end'])
regime_series_full_28 = calculate_regime_series(df_btc_full_28, confirm_n=12)

print(f"\n✅ {len(df_list_full_28)} paires chargées (sur 28 demandées)")
print(f"   Période: 2020-01-01 → 2025-10-03 (couvre tous les cycles)")

In [ ]:
# Paires complètes du live bot (28 paires)
PAIRS_FULL = [
    "BTC/USDT:USDT", "ETH/USDT:USDT", "BNB/USDT:USDT", "SOL/USDT:USDT",
    "XRP/USDT:USDT", "DOGE/USDT:USDT", "ADA/USDT:USDT", "AVAX/USDT:USDT",
    "SHIB/USDT:USDT", "DOT/USDT:USDT", "LINK/USDT:USDT", "MATIC/USDT:USDT",
    "UNI/USDT:USDT", "ATOM/USDT:USDT", "LTC/USDT:USDT", "ETC/USDT:USDT",
    "APT/USDT:USDT", "ARB/USDT:USDT", "OP/USDT:USDT", "NEAR/USDT:USDT",
    "FIL/USDT:USDT", "INJ/USDT:USDT", "IMX/USDT:USDT", "RUNE/USDT:USDT",
    "SUSHI/USDT:USDT", "TRX/USDT:USDT", "AAVE/USDT:USDT", "CRV/USDT:USDT",
]

print(f"📊 PHASE B - VALIDATION PORTFOLIO COMPLET")
print("=" * 80)
print(f"   Paires: {len(PAIRS_FULL)} (portfolio complet)")
print(f"   Période: {PERIODS['train_full']['start']} → {PERIODS['train_full']['end']} (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")
print(f"   Top configs à tester: 3")
print(f"\n⚠️  Cette phase valide que les configs tiennent sur le portfolio complet")
print("=" * 80)

## 3.5️⃣ Phase B - Validation Portfolio complet (28 paires)

Teste le **top-3** des configs sur les **28 paires complètes** (même période train) pour vérifier que les résultats tiennent sur le portfolio complet.

## 4️⃣ Validation Hold-out finale

Test **UNE SEULE FOIS** sur les données de hold-out (2024 H2) pour vérifier qu'il n'y a pas d'overfitting.

In [ ]:
print("\n⚠️  VALIDATION HOLD-OUT FINALE (28 paires)")
print("=" * 80)
print(f"Période: {PERIODS['holdout']['start']} → {PERIODS['holdout']['end']}")
print(f"⚠️  Cette validation ne peut être exécutée qu'UNE SEULE FOIS !\n")

# Filtrer données hold-out (28 paires)
df_list_holdout_28 = filter_df_list_by_dates(df_list_full_28, PERIODS['holdout']['start'], PERIODS['holdout']['end'])
df_btc_holdout = exchange.load_data("BTC/USDT:USDT", "1h", 
                                     start_date=PERIODS['holdout']['start'], 
                                     end_date=PERIODS['holdout']['end'])
regime_holdout = calculate_regime_series(df_btc_holdout, confirm_n=12)

# Utiliser la meilleure config de Phase B (portfolio complet)
best_cfg = df_portfolio.iloc[0]  # Meilleure config selon score Phase B

# Préparer params pour 28 paires
params_coin_holdout = {}
for pair in df_list_holdout_28.keys():
    params_coin_holdout[pair] = {
        "src": "close",
        "ma_base_window": int(best_cfg['ma_window']),
        "envelopes": eval(best_cfg['envelopes']) if isinstance(best_cfg['envelopes'], str) else best_cfg['envelopes'],
        "size": float(best_cfg['size']) / BACKTEST_LEVERAGE
    }

# Test hold-out
if best_cfg['adaptive']:
    adapter_holdout = RegimeBasedAdapter(
        base_params=params_coin_holdout,
        regime_series=regime_holdout,
        regime_params=DEFAULT_PARAMS,
        multipliers={'envelope_std': True},
        base_std=0.10
    )
else:
    adapter_holdout = FixedParamsAdapter(params_coin_holdout)

bt_holdout = run_single_backtest(
    df_list_holdout_28, oldest_pair_28, params_coin_holdout,
    float(best_cfg['stop_loss']), adapter_holdout
)

# Calculer métriques hold-out
holdout_sharpe = bt_holdout.get('sharpe_ratio', 0)
holdout_wallet = bt_holdout['wallet']
holdout_perf = (holdout_wallet / INITIAL_WALLET - 1) * 100
holdout_trades = len(bt_holdout['trades'])

# Comparaison avec Phase B (portfolio)
portfolio_sharpe = best_cfg['sharpe']

print(f"\n📊 RÉSULTATS HOLD-OUT:")
print(f"   Wallet final: ${holdout_wallet:.2f}")
print(f"   Performance: {holdout_perf:+.2f}%")
print(f"   Sharpe Ratio: {holdout_sharpe:.2f}")
print(f"   Nombre de trades: {holdout_trades}")

print(f"\n📊 COMPARAISON:")
print(f"   Phase B Portfolio Sharpe: {portfolio_sharpe:.2f}")
print(f"   Hold-out Sharpe:          {holdout_sharpe:.2f}")

# Validation
sharpe_diff = abs(holdout_sharpe - portfolio_sharpe)

if sharpe_diff <= 0.5:
    print(f"\n✅ VALIDATION RÉUSSIE: Hold-out Sharpe ≈ Portfolio Sharpe (diff={sharpe_diff:.2f})")
    print(f"   Pas d'overfitting détecté. Configuration robuste sur 28 paires.")
elif sharpe_diff <= 1.0:
    print(f"\n⚠️  WARNING: Hold-out Sharpe diverge légèrement (diff={sharpe_diff:.2f})")
    print(f"   Overfitting possible mais acceptable.")
else:
    print(f"\n❌ ÉCHEC: Hold-out Sharpe diverge fortement (diff={sharpe_diff:.2f})")
    print(f"   Overfitting détecté ! Réviser les paramètres.")

print("\n" + "=" * 80)

In [ ]:
# Vérification des données hold-out avant le test
print("🔍 Vérification des données hold-out...")
print(f"Nombre de paires chargées: {len(df_list_holdout_28)}")
for pair, df in list(df_list_holdout_28.items())[:3]:
    print(f"   {pair}: {len(df)} barres ({df.index.min()} → {df.index.max()})")
print(f"\nBTC régime hold-out: {len(regime_holdout)} barres")
print(f"Régimes: {regime_holdout.value_counts().to_dict()}")

## 5️⃣ Résultats et visualisation

In [ ]:
# Scatter plot : Train Sharpe vs Test Sharpe
fig, ax = plt.subplots(figsize=(10, 8))

# Séparer Fixed et Adaptive
df_fixed = df_wf_avg[df_wf_avg['adaptive'] == False]
df_adaptive = df_wf_avg[df_wf_avg['adaptive'] == True]

ax.scatter(df_fixed['train_sharpe'], df_fixed['test_sharpe'], 
           alpha=0.6, s=100, label='Fixed', marker='o')
ax.scatter(df_adaptive['train_sharpe'], df_adaptive['test_sharpe'], 
           alpha=0.6, s=100, label='Adaptive', marker='^')

# Ligne y=x (pas d'overfitting)
max_sharpe = max(df_wf_avg['train_sharpe'].max(), df_wf_avg['test_sharpe'].max())
min_sharpe = min(df_wf_avg['train_sharpe'].min(), df_wf_avg['test_sharpe'].min())
ax.plot([min_sharpe, max_sharpe], [min_sharpe, max_sharpe], 
        'r--', alpha=0.5, label='No overfitting (y=x)')

# Zone acceptable (±0.5)
ax.fill_between([min_sharpe, max_sharpe], 
                 [min_sharpe - 0.5, max_sharpe - 0.5],
                 [min_sharpe + 0.5, max_sharpe + 0.5],
                 alpha=0.1, color='green', label='Acceptable zone (±0.5)')

ax.set_xlabel('Train Sharpe Ratio', fontsize=12)
ax.set_ylabel('Test Sharpe Ratio', fontsize=12)
ax.set_title('Train vs Test Sharpe Ratio (Overfitting Detection)', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'train_vs_test_sharpe_{timestamp}.png', dpi=150)
plt.show()

print(f"💾 Graphique sauvegardé: train_vs_test_sharpe_{timestamp}.png")

In [ ]:
# Bar chart : Top 10 configurations par test score
fig, ax = plt.subplots(figsize=(14, 8))

top10_display = df_wf_avg.head(10).copy()
top10_display['config_label'] = (
    'MA=' + top10_display['ma_window'].astype(str) + 
    ', Size=' + top10_display['size'].astype(str) +
    ', ' + top10_display['adaptive'].map({True: 'Adapt', False: 'Fixed'})
)

x = range(len(top10_display))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], top10_display['train_score'], 
               width, label='Train Score', alpha=0.8, color='steelblue')
bars2 = ax.bar([i + width/2 for i in x], top10_display['test_score'], 
               width, label='Test Score', alpha=0.8, color='coral')

ax.set_xlabel('Configuration', fontsize=12)
ax.set_ylabel('Composite Score', fontsize=12)
ax.set_title('Top 10 Configurations by Test Score', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(top10_display['config_label'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f'top10_scores_{timestamp}.png', dpi=150)
plt.show()

print(f"💾 Graphique sauvegardé: top10_scores_{timestamp}.png")

In [ ]:
# Sauvegarder tous les résultats
df_wf_results.to_csv(f"wf_results_detailed_{timestamp}.csv", index=False)
df_wf_avg.to_csv(f"wf_results_summary_{timestamp}.csv", index=False)

# Sauvegarder meilleure config en JSON
import json

best_config_export = {
    "ma_base_window": int(best_config['ma_window']),
    "envelopes": eval(best_config['envelopes']),
    "size": float(best_config['size']),
    "stop_loss": float(best_config['stop_loss']),
    "adaptive": bool(best_config['adaptive']),
    "train_sharpe": float(best_config['train_sharpe']),
    "test_sharpe": float(best_config['test_sharpe']),
    "test_score": float(best_config['test_score']),
    "holdout_sharpe": float(holdout_sharpe),
    "holdout_perf": float(holdout_perf),
    "timestamp": timestamp,
}

with open(f"best_config_{timestamp}.json", 'w') as f:
    json.dump(best_config_export, f, indent=2)

print(f"\n💾 Résultats sauvegardés:")
print(f"   - wf_results_detailed_{timestamp}.csv (tous les backtests)")
print(f"   - wf_results_summary_{timestamp}.csv (moyennes par config)")
print(f"   - best_config_{timestamp}.json (meilleure configuration)")

## 🎯 Recommandation finale

In [ ]:
print("\n" + "=" * 80)
print("🎯 RECOMMANDATION FINALE")
print("=" * 80)

# Utiliser df_portfolio directement
best_cfg_final = df_portfolio.iloc[0]

print(f"\n✅ Meilleure configuration identifiée (validée sur 28 paires):")
print(f"   Config: {best_cfg_final['config']}")
print(f"   ma_base_window: {best_cfg_final['ma_window']}")
print(f"   envelopes: {best_cfg_final['envelopes']}")
print(f"   size: {best_cfg_final['size']}")
print(f"   stop_loss: {best_cfg_final['stop_loss']}")
print(f"   adaptive: {best_cfg_final['adaptive']}")

print(f"\n📊 Performance validée:")
print(f"   Phase A (8 paires) - Test Score: {top3.iloc[0]['test_score']:.3f}, Sharpe: {top3.iloc[0]['test_sharpe']:.2f}")
print(f"   Phase B (28 paires) - Score: {best_cfg_final['score']:.3f}, Sharpe: {best_cfg_final['sharpe']:.2f}")
print(f"   Hold-out (28 paires) - Sharpe: {holdout_sharpe:.2f}, Perf: {holdout_perf:+.2f}%")

if sharpe_diff <= 0.5:
    print(f"\n✅ Validation: Configuration robuste (pas d'overfitting)")
    print(f"   ✓ Testée sur 8 paires (Walk-Forward)")
    print(f"   ✓ Validée sur 28 paires (Phase B)")
    print(f"   ✓ Hold-out confirmé (2024-H2)")
    print(f"   → RECOMMANDÉ pour mise en production")
elif sharpe_diff <= 1.0:
    print(f"\n⚠️  Validation: Overfitting léger détecté")
    print(f"   → Utiliser avec prudence, surveiller en live")
else:
    print(f"\n❌ Validation: Overfitting significatif")
    print(f"   → NE PAS utiliser en production")
    print(f"   → Réduire la complexité du grid ou augmenter les données")

print(f"\n📝 Prochaines étapes:")
print(f"   1. Appliquer la config dans multi_envelope.ipynb (28 paires)")
print(f"   2. Valider sur paper trading / forward test")
print(f"   3. Si résultats conformes → Déployer en production")

print(f"\n💡 Note:")
print(f"   Cette config a été optimisée sur un échantillon stratifié (8 paires)")
print(f"   puis validée sur le portfolio complet (28 paires) + hold-out.")
print(f"   Méthodologie robuste anti-overfitting.")
print(f"\n⚠️  IMPORTANT: Paramètres GLOBAUX (identiques pour toutes les cryptos)")
print(f"   Pour optimiser par profil (majors/mid-caps/volatiles/low), voir Option 1")

print("\n" + "=" * 80)

In [ ]:
# === CELL-21b : GATE V2 HIÉRARCHIQUE ===

print("="*80)
print("GATE V2 : VALIDATION MULTI-NIVEAUX")
print("="*80)

# Calculer métriques globales
weighted_avg_score = (df_profile_scores['score'] * df_profile_scores['weight']).sum() / df_profile_scores['weight'].sum()
weighted_avg_sharpe = (df_profile_scores['sharpe'] * df_profile_scores['weight']).sum() / df_profile_scores['weight'].sum()

# Métriques de référence (optimisation globale Étape 1)
best_global_score = 2.943
best_global_sharpe = 3.13

# TIER 1 : HARD GATES (doivent passer)
tier1_trades = df_portfolio_total_trades >= 200
tier1_holdout = abs(weighted_avg_sharpe - best_global_sharpe) <= 0.7

tier1_pass = tier1_trades and tier1_holdout

print(f"\nTIER 1 (HARD) :")
print(f"  [{'✅' if tier1_trades else '❌'}] Trades >= 200 : {df_portfolio_total_trades}")
print(f"  [{'✅' if tier1_holdout else '❌'}] |Δ Sharpe holdout| <= 0.7 : {abs(weighted_avg_sharpe - best_global_sharpe):.2f}")

if not tier1_pass:
    print("\n❌ TIER 1 ÉCHOUÉ - Gate rejeté")
    RECOMMENDATION = "global"
else:
    # TIER 2 : SOFT GATES (2 sur 3 suffisent)
    tier2_score = weighted_avg_score > best_global_score
    tier2_sharpe = weighted_avg_sharpe > best_global_sharpe
    tier2_consistency = abs(sharpe_train_avg - sharpe_test_avg) <= 0.5

    tier2_pass = sum([tier2_score, tier2_sharpe, tier2_consistency]) >= 2

    print(f"\nTIER 2 (SOFT - 2/3 requis) :")
    print(f"  [{'✅' if tier2_score else '❌'}] Score > Global : {weighted_avg_score:.2f} vs {best_global_score:.2f}")
    print(f"  [{'✅' if tier2_sharpe else '❌'}] Sharpe > Global : {weighted_avg_sharpe:.2f} vs {best_global_sharpe:.2f}")
    print(f"  [{'✅' if tier2_consistency else '❌'}] |Δ Sharpe train-test| <= 0.5 : {abs(sharpe_train_avg - sharpe_test_avg):.2f}")
    print(f"  → Passed: {sum([tier2_score, tier2_sharpe, tier2_consistency])}/3")

    # TIER 3 : WARNING (log only)
    tier3_phase = abs(sharpe_phaseA - sharpe_phaseB) <= 0.5

    print(f"\nTIER 3 (WARNING) :")
    print(f"  [{'✅' if tier3_phase else '⚠️ '}] |Δ Sharpe Phase A-B| <= 0.5 : {abs(sharpe_phaseA - sharpe_phaseB):.2f}")

    # Décision finale
    if tier2_pass:
        RECOMMENDATION = "profil"
        print("\n✅ GATE V2 VALIDÉ - Utiliser optimisation par profils")
    else:
        RECOMMENDATION = "global"
        print("\n❌ GATE V2 ÉCHOUÉ - Utiliser optimisation globale")

print("="*80)
print(f"RECOMMANDATION FINALE : {RECOMMENDATION.upper()}")
print("="*80)

# Sauvegarder résultats
results_final = {
    'recommendation': RECOMMENDATION,
    'weighted_score_profile': weighted_avg_score,
    'weighted_sharpe_profile': weighted_avg_sharpe,
    'score_global': best_global_score,
    'sharpe_global': best_global_sharpe,
    'tier1_pass': tier1_pass,
    'tier2_pass': tier2_pass if tier1_pass else False,
    'tier3_pass': tier3_phase,
    'timestamp': pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
}

# Exporter
import json
output_file = f"gate_v2_result_{results_final['timestamp']}.json"
with open(output_file, 'w') as f:
    json.dump(results_final, f, indent=2)

print(f"\n✅ Résultats sauvegardés : {output_file}")